## Transform data into star schema

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType


Load Fact Payment Table into fact_payments

In [0]:
payments = spark.table("default.staging_payments")

In [0]:
display(payments.limit(10))

payment_id,date,amount,rider_id
1,2019-05-01,9.0,1000
2,2019-06-01,9.0,1000
3,2019-07-01,9.0,1000
4,2019-08-01,9.0,1000
5,2019-09-01,9.0,1000
6,2019-10-01,9.0,1000
7,2019-11-01,9.0,1000
8,2019-12-01,9.0,1000
9,2020-01-01,9.0,1000
10,2020-02-01,9.0,1000


In [0]:
# Write data to fact_payments
spark.sql("drop table if exists default.fact_payments")
payments.dropDuplicates(["payment_id"]).write.format("delta").mode("overwrite").saveAsTable("default.fact_payments")

Load Riders Table into dim_riders

In [0]:
riders = spark.table("default.staging_riders")

In [0]:
display(riders.limit(10))

rider_id,first_name,last_name,address,birthday,account_start_date,account_end_date,is_member
1000,Diana,Clark,1200 Alyssa Squares,1989-02-13,2019-04-23,null,true
1001,Jennifer,Smith,397 Diana Ferry,1976-08-10,2019-11-01,2020-09-01,true
1002,Karen,Smith,644 Brittany Row Apt. 097,1998-08-10,2022-02-04,null,true
1003,Bryan,Roberts,996 Dickerson Turnpike,1999-03-29,2019-08-26,null,false
1004,Jesse,Middleton,7009 Nathan Expressway,1969-04-11,2019-09-14,null,true
1005,Christine,Rodriguez,224 Washington Mills Apt. 467,1974-08-27,2020-03-24,null,false
1006,Alicia,Taylor,1137 Angela Locks,2004-01-30,2020-11-27,2021-12-01,true
1007,Benjamin,Fernandez,979 Phillips Ways,1988-01-11,2016-12-11,null,false
1008,John,Crawford,7691 Evans Court,1987-02-21,2021-03-28,2021-07-01,true
1009,Victoria,Ritter,9922 Jim Crest Apt. 319,1981-02-07,2020-06-12,2021-11-01,true


In [0]:
# Write data to dim_rider
spark.sql("drop table if exists default.dim_rider")
riders.dropDuplicates(["rider_id"]).write.format("delta").mode("overwrite").saveAsTable("default.dim_rider")

Load Stations Table into dim_stations

In [0]:
stations = spark.table("default.staging_stations")


In [0]:
display(stations.limit(10))

station_id,name,latitude,longitude
525,Glenwood Ave & Touhy Ave,42.012701,-87.66605799999999
KA1503000012,Clark St & Lake St,41.88579466666667,-87.63110066666668
637,Wood St & Chicago Ave,41.895634,-87.672069
13216,State St & 33rd St,41.8347335,-87.6258275
18003,Fairbanks St & Superior St,41.89580766666667,-87.62025316666669
KP1705001026,LaSalle Dr & Huron St,41.894877,-87.632326
13253,Lincoln Ave & Waveland Ave,41.948797,-87.675278
KA1503000044,Rush St & Hubbard St,41.890173,-87.62618499999999
KA1504000140,Winchester Ave & Elston Ave,41.92403733333333,-87.67641483333334
TA1305000032,Clinton St & Madison St,41.882242,-87.64106600000001


In [0]:
# Write data to dim_station
spark.sql("drop table if exists default.dim_station")
stations.dropDuplicates(["station_id"]).write.format("delta").mode("overwrite").saveAsTable("default.dim_station")

Load Trip Table into fact_trips

In [0]:
trips = spark.table("default.staging_trips")


In [0]:
display(trips.limit(10))

trip_id,rideable_type,start_at,ended_at,start_station_id,end_station_id,rider_id
start_date89E7AA6C29227EFF,classic_bike,2021-02-12T16:14:56.000+0000,2021-02-12T16:21:43.000+0000,525,660,71934
0FEFDE2603568365,classic_bike,2021-02-14T17:52:38.000+0000,2021-02-14T18:12:09.000+0000,525,16806,47854
E6159D746B2DBB91,electric_bike,2021-02-09T19:10:18.000+0000,2021-02-09T19:19:10.000+0000,KA1503000012,TA1305000029,70870
B32D3199F1C2E75B,classic_bike,2021-02-02T17:49:41.000+0000,2021-02-02T17:54:06.000+0000,637,TA1305000034,58974
83E463F23575F4BF,electric_bike,2021-02-23T15:07:23.000+0000,2021-02-23T15:22:37.000+0000,13216,TA1309000055,39608
BDAA7E3494E8D545,electric_bike,2021-02-24T15:43:33.000+0000,2021-02-24T15:49:05.000+0000,18003,KP1705001026,36267
A772742351171257,classic_bike,2021-02-01T17:47:42.000+0000,2021-02-01T17:48:33.000+0000,KP1705001026,KP1705001026,50104
295476889D9B79F8,classic_bike,2021-02-11T18:33:53.000+0000,2021-02-11T18:35:09.000+0000,18003,18003,19618
362087194BA4CC9A,classic_bike,2021-02-27T15:13:39.000+0000,2021-02-27T15:36:36.000+0000,KP1705001026,KP1705001026,16732
21630F715038CCB0,classic_bike,2021-02-20T08:59:42.000+0000,2021-02-20T09:17:04.000+0000,KP1705001026,KP1705001026,57068


Load Trips Table into fact_trips

In [0]:
fact_trips = trips.alias("tbl1").join(riders.alias("tbl2"), col("tbl1.rider_id") == col("tbl2.rider_id"), "left") \
            .withColumn('duration', round((unix_timestamp("ended_at") - unix_timestamp('start_at'))/60)) \
            .withColumn('rider_age', round((unix_timestamp("account_start_date") - unix_timestamp('birthday'))/3600/24)) \
            .withColumnRenamed('start_at', 'start_time_id') \
            .withColumnRenamed('ended_at', 'end_time_id') \
            .select("trip_id", col("tbl1.rider_id"), "rideable_type", "start_station_id", "end_station_id", "start_time_id", "end_time_id", "duration", "rider_age")
            
display(fact_trips.limit(10))

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/python_shell/dbruntime/lsp_backend/lsp_manager.py", line 243, in _monitor
    self.restart(monitor=False)
  File "/databricks/python_shell/dbruntime/lsp_backend/lsp_manager.py", line 224, in restart
    self.stop(monitor=monitor)
  File "/databricks/python_shell/dbruntime/lsp_backend/lsp_manager.py", line 203, in stop
    self.write_stream.close()
  File "/databricks/python/lib/python3.9/site-packages/pylsp_jsonrpc/streams.py", line 90, in close
    self._wfile.close()
BrokenPipeError: [Errno 32] Broken pipe


trip_id,rider_id,rideable_type,start_station_id,end_station_id,start_time_id,end_time_id,duration,rider_age
start_date89E7AA6C29227EFF,71934,classic_bike,525,660,2021-02-12T16:14:56.000+0000,2021-02-12T16:21:43.000+0000,7.0,13638.0
0FEFDE2603568365,47854,classic_bike,525,16806,2021-02-14T17:52:38.000+0000,2021-02-14T18:12:09.000+0000,20.0,13523.0
E6159D746B2DBB91,70870,electric_bike,KA1503000012,TA1305000029,2021-02-09T19:10:18.000+0000,2021-02-09T19:19:10.000+0000,9.0,12051.0
B32D3199F1C2E75B,58974,classic_bike,637,TA1305000034,2021-02-02T17:49:41.000+0000,2021-02-02T17:54:06.000+0000,4.0,7107.0
83E463F23575F4BF,39608,electric_bike,13216,TA1309000055,2021-02-23T15:07:23.000+0000,2021-02-23T15:22:37.000+0000,15.0,25961.0
BDAA7E3494E8D545,36267,electric_bike,18003,KP1705001026,2021-02-24T15:43:33.000+0000,2021-02-24T15:49:05.000+0000,6.0,10176.0
A772742351171257,50104,classic_bike,KP1705001026,KP1705001026,2021-02-01T17:47:42.000+0000,2021-02-01T17:48:33.000+0000,1.0,10697.0
295476889D9B79F8,19618,classic_bike,18003,18003,2021-02-11T18:33:53.000+0000,2021-02-11T18:35:09.000+0000,1.0,7767.0
362087194BA4CC9A,16732,classic_bike,KP1705001026,KP1705001026,2021-02-27T15:13:39.000+0000,2021-02-27T15:36:36.000+0000,23.0,5626.0
21630F715038CCB0,57068,classic_bike,KP1705001026,KP1705001026,2021-02-20T08:59:42.000+0000,2021-02-20T09:17:04.000+0000,17.0,16461.0


In [0]:
# Write data to fact_trip
spark.sql("drop table if exists default.fact_trip")
fact_trips.write.format("delta").mode("overwrite").saveAsTable("default.fact_trip")

Create Dim_time

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.types import StringType
# Get min date from trips
min_date = trips.selectExpr('MIN(start_at) AS started_at').first().asDict()['started_at']
# Add date range as max start_at from trip plus 5 years
max_date = trips.selectExpr('DATEADD(year, 5, MAX(start_at)) AS started_at').first().asDict()['started_at']
expression = f"sequence(to_date('{min_date}'), to_date('{max_date}'), interval 1 day)"
dim_time = spark.createDataFrame([(1,)], ["time_id"])

dim_time = dim_time.withColumn("dateinit", f.explode(f.expr(expression)))
dim_time = dim_time.withColumn("date", f.to_timestamp(dim_time.dateinit, "yyyy-MM-dd"))

dim_time = dim_time \
            .withColumn("day_of_week", f.dayofweek(dim_time.date)) \
            .withColumn("day_of_month", f.dayofmonth(dim_time.date)) \
            .withColumn("week_of_year", f.weekofyear(dim_time.date)) \
            .withColumn("year", f.year(dim_time.date)) \
            .withColumn("quarter", f.quarter(dim_time.date)) \
            .withColumn("month", f.month(dim_time.date)) \
            .withColumn("date", dim_time.date.cast(StringType())) \
            .drop(f.col("dateinit"))

In [0]:
# Write data to dim_time
spark.sql("drop table if exists default.dim_time")
dim_time.write.format("delta").mode("overwrite").saveAsTable("default.dim_time")